In [29]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from keras.optimizers import Adam
from keras.models import load_model

In [30]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
train_gen = train_datagen.flow_from_directory('/home/andy/Desktop/image_datasets/train',target_size=(56,56),batch_size=32,class_mode='categorical')
val_gen = test_datagen.flow_from_directory("/home/andy/Desktop/image_datasets/test",target_size=(56,56),batch_size=32,class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [40]:
train_gen.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [41]:
base_path = "/home/andy/Desktop/image_datasets/"
distribution_dict={}
for dir_ in os.listdir(os.path.join(base_path,'train')):
    distribution_dict[dir_] = len([name for name in os.listdir(os.path.join((os.path.join(base_path,'train')),dir_))])

In [42]:
distribution_dict

{'happy': 7215,
 'disgust': 436,
 'angry': 3995,
 'surprise': 3171,
 'neutral': 4965,
 'fear': 4097,
 'sad': 4830}

In [43]:
init_lr = 1e-4
epochs =50
opt = Adam(lr=init_lr,decay=init_lr/epochs)

In [44]:
model = load_model("/home/andy/Desktop/image_models/resnet_emotion/")

In [45]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 62, 62, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 28, 28, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 28, 28, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [46]:
for layer in model.layers:
    if layer.name == 'avg_pool':
        break
    layer.trainable = False

In [47]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 62, 62, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 28, 28, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 28, 28, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [48]:
import keras

In [49]:
model.compile(optimizer=opt,
             loss={
                 
                 "emotion": 'categorical_crossentropy'
             },
             loss_weights={
                 
                 "emotion": 1.5
             },
             metrics={
                 
                 "emotion": "categorical_accuracy"
             })

In [50]:
from keras.callbacks import ModelCheckpoint

In [51]:
call = [ModelCheckpoint('/home/andy/Desktop/image_models/resnet_trained_checkpoint',save_freq='epoch',save_weights_only=True,monitor='val_loss')]

In [55]:
history = model.fit_generator(generator=train_gen,
                             steps_per_epoch=28709//32,
                             epochs=50,
                             validation_data=val_gen,
                             validation_steps=7178//32)

Epoch 1/50
897/897 [==============================] - 110s 122ms/step - loss: 2.9648 - categorical_accuracy: 0.2588 - val_loss: 2.8555 - val_categorical_accuracy: 0.3001
Epoch 2/50
897/897 [==============================] - 111s 124ms/step - loss: 2.9186 - categorical_accuracy: 0.2711 - val_loss: 2.8289 - val_categorical_accuracy: 0.2985
Epoch 3/50
897/897 [==============================] - 108s 121ms/step - loss: 2.8867 - categorical_accuracy: 0.2753 - val_loss: 2.7844 - val_categorical_accuracy: 0.3090
Epoch 4/50
897/897 [==============================] - 107s 120ms/step - loss: 2.8486 - categorical_accuracy: 0.2860 - val_loss: 2.7539 - val_categorical_accuracy: 0.3192
Epoch 5/50
897/897 [==============================] - 108s 120ms/step - loss: 2.8230 - categorical_accuracy: 0.2931 - val_loss: 2.8961 - val_categorical_accuracy: 0.2977
Epoch 6/50
897/897 [==============================] - 108s 120ms/step - loss: 2.8050 - categorical_accuracy: 0.2961 - val_loss: 2.7997 - val_categoric

Epoch 49/50
897/897 [==============================] - 106s 119ms/step - loss: 2.5011 - categorical_accuracy: 0.3460 - val_loss: 2.5607 - val_categorical_accuracy: 0.3133
Epoch 50/50
897/897 [==============================] - 107s 119ms/step - loss: 2.5096 - categorical_accuracy: 0.3408 - val_loss: 2.4619 - val_categorical_accuracy: 0.3585
